# Batch Processing: Summarize Support Tickets with an LLM

This notebook demonstrates a practical use case for a deployed Large Language Model: batch processing. We will simulate a scenario where we have a folder of support tickets (in JSON format) and use our model to automatically:

1.  Read each ticket.
2.  Generate a concise, one-sentence summary of the issue.
3.  Suggest potential troubleshooting steps.
4.  Save this analysis into a new summary file.

This workflow can significantly speed up ticket triage and analysis for a support team.

## 1. Install Required Libraries

First, we'll ensure the necessary Python libraries are installed. `langchain-openai` helps us communicate with the model, and `httpx` is the HTTP client that handles the requests.

In [ ]:
%pip install -q langchain-openai httpx

## 2. Imports and Configuration

Next, we'll import the libraries and configure the connection to our deployed model. 

**Action Required:**
1.  **`BASE_URL`**: Replace the placeholder with the **Inference endpoint** URL from your model's details page.
2.  **`API_KEY`**: Replace the placeholder with the **Authentication Token** from the 'Authentication' section of the model's details page.
3.  **`MODEL_NAME`**: Ensure this matches the name of your model deployment.

In [ ]:
import os
import json
import httpx
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# --- ❗ACTION REQUIRED: REPLACE VALUES BELOW ❗---

# Model and Connection Details
MODEL_NAME = "granite" # The name of your model deployment
BASE_URL = "https://your-model-inference-url.apps.cluster.com/v1" # Replace with your Inference Endpoint
API_KEY = "sha256~your-long-authentication-token" # Replace with your Authentication Token

# -----------------------------------------------------

# Directory and File Handling Configuration
INPUT_FOLDER = "support_tickets"
# This is the key in the JSON file that contains the main ticket text.
JSON_TICKET_KEY = "issue_description"

## 3. Prepare Sample Data

For this notebook to be self-contained, we will programmatically create the `support_tickets` directory and populate it with a few sample JSON files. In a real-world scenario, this folder would already exist and be populated with tickets.

In [ ]:
# Create the input folder if it doesn't exist
os.makedirs(INPUT_FOLDER, exist_ok=True)

# Define sample ticket data
ticket1_data = {
    "ticket_id": "TICK-001",
    "customer_name": "Alice",
    "issue_description": "I'm trying to log into the main dashboard, but after I enter my password, the page just spins and never loads. I've tried clearing my browser cache and using a different browser (Chrome and Firefox), but the issue persists. My colleague can log in just fine."
}

ticket2_data = {
    "ticket_id": "TICK-002",
    "customer_name": "Bob",
    "issue_description": "The monthly sales report is failing to generate. When I click the 'Export to PDF' button, I receive a generic 'Error 500 - Internal Server Error' message. This was working yesterday. I need this report for a meeting this afternoon."
}

# Write the sample tickets to JSON files
with open(os.path.join(INPUT_FOLDER, "ticket_001.json"), 'w') as f:
    json.dump(ticket1_data, f, indent=2)

with open(os.path.join(INPUT_FOLDER, "ticket_002.json"), 'w') as f:
    json.dump(ticket2_data, f, indent=2)

print(f"Successfully created sample files in the '{INPUT_FOLDER}' directory.")

## 4. Initialize the LLM Client

Now, we set up the `ChatOpenAI` client with our connection details. We'll set a low `temperature` to encourage the model to provide more factual, less creative responses, which is ideal for a support context. 

**Note:** We are using `verify=False` to disable SSL verification, which is often necessary in development or sandboxed OpenShift environments. This is not recommended for production.

In [ ]:
llm = ChatOpenAI(
    model=MODEL_NAME,
    api_key=API_KEY,
    base_url=BASE_URL,
    http_client=httpx.Client(verify=False),
    temperature=0.1,  # Lower temperature for more factual responses
)

print("LLM client initialized.")

## 5. Define the Core Processing Function

This function takes the text from a single support ticket, wraps it in a carefully crafted prompt, and sends it to the LLM. The prompt instructs the model to act as a senior support engineer, providing both a summary and potential solutions.

In [ ]:
def analyze_and_summarize_ticket(ticket_content):
    """
    Sends ticket content to the LLM for summarization and solution generation.
    """
    # Define a clear, instructional prompt for the model
    prompt = f"""
    You are a senior support engineer. Please perform the following tasks for the support ticket below:
    1. Provide a concise one-sentence summary of the user's issue.
    2. Based on the issue, list 3 to 4 potential troubleshooting steps or solutions in a numbered list.

    Here is the support ticket:
    ---
    {ticket_content}
    ---
    """

    messages = [
        SystemMessage(content="You are a helpful senior support engineer."),
        HumanMessage(content=prompt),
    ]

    try:
        ai_msg = llm.invoke(messages)
        return ai_msg.content
    except Exception as e:
        return f"Error communicating with the model: {e}"

## 6. Run the Batch Process

This is the main execution logic. The code will:
1. Find all `.json` files in our `support_tickets` directory.
2. Loop through each file.
3. Read the content and extract the ticket description.
4. Call our `analyze_and_summarize_ticket` function.
5. Save the model's response to a new `_summary.txt` file.

In [ ]:
# Find all JSON files in the directory
json_files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith('.json')]

if not json_files:
    print(f"No JSON files found in the '{INPUT_FOLDER}' directory.")
else:
    print(f"Found {len(json_files)} support tickets to process...")
    for file_name in json_files:
        json_path = os.path.join(INPUT_FOLDER, file_name)
        print(f"\nProcessing: {file_name}")

        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            # Extract the ticket text using the specified key
            ticket_text = data.get(JSON_TICKET_KEY)

            if not ticket_text:
                print(f"  - Skipping: Could not find key '{JSON_TICKET_KEY}' in the file.")
                continue

            # Get the summary and solutions from the model
            summary_and_solutions = analyze_and_summarize_ticket(ticket_text)

            # Define the output filename
            base_name = os.path.splitext(file_name)[0]
            output_filename = f"{base_name}_summary.txt"
            output_path = os.path.join(INPUT_FOLDER, output_filename)

            # Save the result to a text file
            with open(output_path, 'w') as out_f:
                out_f.write(summary_and_solutions)

            print(f"  - Successfully saved summary to: {output_filename}")

        except json.JSONDecodeError:
            print(f"  - Skipping: Invalid JSON format in {file_name}.")
        except Exception as e:
            print(f"  - An unexpected error occurred: {e}")

    print("\nProcessing complete.")

## 7. Verify the Output

Finally, let's check our work. The code below will list all the files in the `support_tickets` directory (you should see the new `_summary.txt` files) and then print the content of the first summary file.

In [ ]:
print(f"--- Contents of '{INPUT_FOLDER}' directory ---")
for file in os.listdir(INPUT_FOLDER):
    print(file)
print("----------------------------------------\n")

# Print the content of the first summary file
summary_file_path = os.path.join(INPUT_FOLDER, "ticket_001_summary.txt")
if os.path.exists(summary_file_path):
    print(f"--- Content of {os.path.basename(summary_file_path)} ---")
    with open(summary_file_path, 'r') as f:
        print(f.read())
    print("----------------------------------------")
